This code comes from here:
https://github.com/vinay20045/json-to-csv/blob/master/json_to_csv.py

Edit the year you want to convert below

In [2]:
import pandas as pd

xplore_numbers = pd.read_csv("xplore-isnumbers.csv", dtype=object) #readings all as strings
print(xplore_numbers.head(60))


    year isnumber    conference     json_filename
0   1990     3914      Vis-conf      articles-vis
1   1991     4467      Vis-conf      articles-vis
2   1992     6054      Vis-conf      articles-vis
3   1993     9012      Vis-conf      articles-vis
4   1994     8039      Vis-conf      articles-vis
5   1995    10213      Vis-conf      articles-vis
6   1995    11604  InfoVis-conf  articles-infovis
7   1996    12277      Vis-conf      articles-vis
8   1996    12180  InfoVis-conf  articles-infovis
9   1997    14360      Vis-conf      articles-vis
10  1997    13801  InfoVis-conf  articles-infovis
11  1998    16079      Vis-conf      articles-vis
12  1998    15744  InfoVis-conf  articles-infovis
13  1999    17553      Vis-conf      articles-vis
14  1999    17394  InfoVis-conf  articles-infovis
15  2000    19150      Vis-conf      articles-vis
16  2000    19138  InfoVis-conf  articles-infovis
17  2001    20824      Vis-conf      articles-vis
18  2001    20791  InfoVis-conf  articles-infovis


In [3]:
import sys
import json
import csv

##
# Convert to string keeping encoding in mind...
##
def to_string(s):
    try:
        return str(s)
    except:
        #Change the encoding type if needed
        return s.encode('utf-8')


##
# This function converts an item like 
# {
#   "item_1":"value_11",
#   "item_2":"value_12",
#   "item_3":"value_13",
#   "item_4":["sub_value_14", "sub_value_15"],
#   "item_5":{
#       "sub_item_1":"sub_item_value_11",
#       "sub_item_2":["sub_item_value_12", "sub_item_value_13"]
#   }
# }
# To
# {
#   "node_item_1":"value_11",
#   "node_item_2":"value_12",
#   "node_item_3":"value_13",
#   "node_item_4_0":"sub_value_14", 
#   "node_item_4_1":"sub_value_15",
#   "node_item_5_sub_item_1":"sub_item_value_11",
#   "node_item_5_sub_item_2_0":"sub_item_value_12",
#   "node_item_5_sub_item_2_0":"sub_item_value_13"
# }
##
def reduce_item(key, value):
    global reduced_item
    
    #Reduction Condition 1
    if type(value) is list:
        i=0
        for sub_item in value:
            reduce_item(key+'_'+to_string(i), sub_item)
            i=i+1

    #Reduction Condition 2
    elif type(value) is dict:
        sub_keys = value.keys()
        for sub_key in sub_keys:
            reduce_item(key+'_'+to_string(sub_key), value[sub_key])
    
    #Base Condition
    else:
        reduced_item[to_string(key)] = to_string(value)



In [5]:
#Reading arguments
import os

for index, row in xplore_numbers.iterrows():
    year = row['year']
    
    
    if not pd.isnull(row['json_filename']):
        node = "articles"
        
        generated_data_path = year +"/generated_data/"
        pathExists = os.path.exists(generated_data_path)
        if not pathExists:
            os.makedirs(generated_data_path)
        
        json_file_path = year + "/" + row['json_filename']+".json"
        csv_file_path = generated_data_path + row['json_filename']+".csv"
        
        print ("Now working on "+csv_file_path)
        
        #TODO: We could check if the .csv file already exists and then decide not to do anything

        #fp = open(json_file_path, 'r',encoding="utf8")
        fp = open(json_file_path, mode = 'rb')
        json_value = fp.read()
        raw_data = json.loads(json_value)

        try:
            data_to_be_processed = raw_data[node]
        except:
            data_to_be_processed = raw_data

        processed_data = []
        header = []
        for item in data_to_be_processed:
            reduced_item = {}
            reduce_item(node, item)

            header += reduced_item.keys()

            processed_data.append(reduced_item)

        header = list(set(header))
        header.sort()

        with open(csv_file_path, 'w+',encoding="utf8") as f:
            writer = csv.DictWriter(f, header, quoting=csv.QUOTE_ALL)
            writer.writeheader()
            for row in processed_data:
                writer.writerow(row)

        print ("Just completed writing "+csv_file_path+" file with %d columns" % len(header))
        
    else:
        print("We are missing a filename for: " + year + " " + row['conference']+ ". Did you run Step 0 already?")



Now working on 1990/generated_data/articles-vis.csv
Just completed writing 1990/generated_data/articles-vis.csv file with 65 columns
Now working on 1991/generated_data/articles-vis.csv
Just completed writing 1991/generated_data/articles-vis.csv file with 70 columns
Now working on 1992/generated_data/articles-vis.csv
Just completed writing 1992/generated_data/articles-vis.csv file with 79 columns
Now working on 1993/generated_data/articles-vis.csv
Just completed writing 1993/generated_data/articles-vis.csv file with 66 columns
Now working on 1994/generated_data/articles-vis.csv
Just completed writing 1994/generated_data/articles-vis.csv file with 76 columns
Now working on 1995/generated_data/articles-vis.csv
Just completed writing 1995/generated_data/articles-vis.csv file with 70 columns
Now working on 1995/generated_data/articles-infovis.csv
Just completed writing 1995/generated_data/articles-infovis.csv file with 71 columns
Now working on 1996/generated_data/articles-vis.csv
Just comp